# Lectura 27: DataFrame - Manipulación y selección III

## `sort`

Esta función ordena el DataFrame por la(s) columna(s) proporcionadas.

In [ ]:
import polars as pl

vuelos = pl.read_parquet('./data/vuelos/', use_pyarrow=True)

vuelos

In [ ]:
vuelos.sort('AIR_TIME')

Como se puede observar el orden por defecto es ascendente y los nulos han sido colocados de primeros. Si deseamos que los valores nulos sean colocados al final podemos emplear el parámetro `nulls_last=True`.

In [ ]:
vuelos.sort('AIR_TIME', nulls_last=True)

Si deseamos cambair el tipo de orden a descendiente podemos emplear el parámetro `descending=True`.

In [ ]:
vuelos.sort('AIR_TIME', descending=True, nulls_last=True)

Si deseamos ordenar por más de una columna podemos pasarle las columnas a la función `sort` en una lista.

In [ ]:
vuelos.sort(['AIR_TIME', 'DISTANCE'], descending=True, nulls_last=True)

Si para este último caso deseamos indicarle un orden específico (ascendente o descendente) para cada columna podemos pasarle al parámetro `descending` una lista con los valores `True` o `False` indicándole el tipo de orden de cada columna.

In [ ]:
vuelos.sort(['AIR_TIME', 'DISTANCE'], descending=[True, False], nulls_last=True)

## `join`

Esta función realiza el join entre dos DataFrames similar a como lo haría en SQL.

In [ ]:
departamentos = pl.read_parquet('./data/departamentos/', use_pyarrow=True)

empleados = pl.read_parquet('./data/empleados/', use_pyarrow=True)

print(departamentos)

print(empleados)

In [ ]:
empleados.join(departamentos, left_on='num_dpto', right_on='id', how='inner')

Vamos a renombrar la columna `num_dpto` del DataFrame `empleados` a `id`.

In [ ]:
from polars import col

empleados = empleados.rename({'num_dpto': 'id'})

Si deseamos realizar un join entre dos DataFrames por una columna que tiene el mismo nombre en ambos DataFrame utilizamos el parámetro `on=colName`.

In [ ]:
empleados.join(departamentos, on='id', how='left')

In [ ]:
empleados.join(departamentos, on='id', how='anti')

In [ ]:
empleados.join(departamentos, on='id', how='semi')

## `pivot`

Esta función permite crear agregaciones por grupos con pivoteo de alguna columna.

Para mostrar como trabaja esta función vamos a leer el DataFrame de estudiantes y vamos a responder la siguiente pregunta:

¿Cuál fue el promedio de peso de los estudiantes por año de graduación y sexo?

In [ ]:
estudiantes = pl.read_parquet('./data/pivoteo/', use_pyarrow=True)

estudiantes

Los principales parámetros son los siguientes:
- `index`: columna por la cual deseamos agrupar
- `columns`: columna por la cual deseamos realizar el pivoteo
- `values`: columna a la cual deseamos realizarle la agregación
- `aggregate_function`: función de agregación que se desea aplicar
    - Este parámetro admite las siguientes opciones: `min`, `max`, `first`, `last`, `sum`, `mean`, `median` y `len`

In [ ]:
estudiantes.pivot(
    index='graduacion',
    columns='sexo',
    values='peso',
    aggregate_function='mean'
)

## `to_dummies`

Esta función convierte una columna categórica en variables indicadoras. Si no se le proporciona ningún argumento convertirá todas las columnas del DataFrame a dummies.

In [ ]:
estudiantes

In [ ]:
estudiantes.to_dummies('sexo')

Si deseamos eliminar la primera categoría de la columna que se está convirtiendo a dummy podemos emplear el parámetro `drop_first=True`.

In [ ]:
estudiantes.to_dummies('sexo', drop_first=True)

## `rechunk` y `shrink_to_fit`

Por último vamos a ver dos funciones que nos ayudarán a optimizar el desempeño de las consultas que se ejecuten sobre un DataFrame y a optimizar la memoria utilizada por un DataFrame.

### `rechunk`

Esta función hace un "rechunk" de los datos del DataFrame para colocarlos en una asignación contigua de memoria.

Esto garantizará que todas las operaciones posteriores tengan un rendimiento óptimo y predecible.

In [ ]:
estudiantes.rechunk

### `shrink_to_fit`

Esta función reduce el uso de memoria del DataFrame.

El DataFrame se encoge para ajustarse a la capacidad exacta necesaria para contener los datos.

In [ ]:
estudiantes.shrink_to_fit